### Write testutility file

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


### Write YAML file

In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: housingPricesPaid 
file_name: price_paid_records
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Transaction unique identifier
    - Price
    - Date of Transfer
    - Property Type
    - Old/New
    - Duration
    - Town/City
    - District
    - County
    - PPDCategory Type
    - Record Status - monthly file only

Overwriting file.yaml


### Installing YAML

In [10]:
pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


### Reading config file

In [3]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [4]:
config_data['inbound_delimiter']

','

In [5]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'housingPricesPaid',
 'file_name': 'price_paid_records',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Transaction unique identifier',
  'Price',
  'Date of Transfer',
  'Property Type',
  'Old/New',
  'Duration',
  'Town/City',
  'District',
  'County',
  'PPDCategory Type',
  'Record Status - monthly file only']}

### Reading the data

In [1]:
# Normal reading process of the file
import pandas as pd

path = r"C:\Users\sefaa\OneDrive\Desktop\DataGlacier-VC\Task6"    

df = pd.read_csv(path+"/price_paid_records.csv", delimiter=",")
df.head()

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [4]:
# read the file using config file
import pandas as pd
path = r"C:\Users\sefaa\OneDrive\Desktop\DataGlacier-VC\Task6"  
file_type = config_data['file_type']
source_file = path + "./" + config_data['file_name'] + f'.{file_type}'

#print("",source_file)
df_config = pd.read_csv(source_file,config_data['inbound_delimiter'])
df_config.head()

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


### Checking from null values

In [39]:
print(df.isnull().sum().sum())

0


In [7]:
print(df_config.isnull().sum().sum())

0


### Getting columns names

In [40]:
# iterating the columns
for col in df.columns:
    print(col)

Transaction unique identifier
Price
Date of Transfer
Property Type
Old/New
Duration
Town/City
District
County
PPDCategory Type
Record Status - monthly file only


### Validation

In [9]:
#validate the header of the file
util.col_header_val(df_config,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['record_status_monthly_file_only', 'date_of_transfer', 'transaction_unique_identifier', 'property_type', 'ppdcategory_type', 'town_city', 'old_new']
Following YAML columns are not in the file uploaded ['record status - monthly file only', 'transaction unique identifier', 'date of transfer', 'old/new', 'ppdcategory type', 'town/city', 'property type']


0

In [10]:
print("columns of files are:" ,df_config.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['transaction_unique_identifier', 'price', 'date_of_transfer',
       'property_type', 'old_new', 'duration', 'town_city', 'district',
       'county', 'ppdcategory_type', 'record_status_monthly_file_only'],
      dtype='object')
columns of YAML are: ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']


In [11]:
if util.col_header_val(df_config,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['record_status_monthly_file_only', 'date_of_transfer', 'transaction_unique_identifier', 'property_type', 'ppdcategory_type', 'town_city', 'old_new']
Following YAML columns are not in the file uploaded ['record status - monthly file only', 'transaction unique identifier', 'date of transfer', 'old/new', 'ppdcategory type', 'town/city', 'property type']
validation failed


In [1]:
import pandas as pd
path = r"C:\Users\sefaa\OneDrive\Desktop\DataGlacier-VC\Task6" 

In [2]:
pd.read_csv(path+"/price_paid_records.csv")

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...
22489343,{4C4EE000-291A-1854-E050-A8C063054F34},175000,2017-02-20 00:00,S,N,F,LEEDS,LEEDS,WEST YORKSHIRE,A,A
22489344,{4C4EE000-291B-1854-E050-A8C063054F34},586945,2017-02-15 00:00,D,N,F,WETHERBY,LEEDS,WEST YORKSHIRE,A,A
22489345,{4C4EE000-291C-1854-E050-A8C063054F34},274000,2017-02-24 00:00,D,N,L,HUDDERSFIELD,KIRKLEES,WEST YORKSHIRE,A,A
22489346,{4C4EE000-291D-1854-E050-A8C063054F34},36000,2017-02-22 00:00,T,N,F,HALIFAX,CALDERDALE,WEST YORKSHIRE,A,A


In [5]:
df_config

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...
22489343,{4C4EE000-291A-1854-E050-A8C063054F34},175000,2017-02-20 00:00,S,N,F,LEEDS,LEEDS,WEST YORKSHIRE,A,A
22489344,{4C4EE000-291B-1854-E050-A8C063054F34},586945,2017-02-15 00:00,D,N,F,WETHERBY,LEEDS,WEST YORKSHIRE,A,A
22489345,{4C4EE000-291C-1854-E050-A8C063054F34},274000,2017-02-24 00:00,D,N,L,HUDDERSFIELD,KIRKLEES,WEST YORKSHIRE,A,A
22489346,{4C4EE000-291D-1854-E050-A8C063054F34},36000,2017-02-22 00:00,T,N,F,HALIFAX,CALDERDALE,WEST YORKSHIRE,A,A


In [12]:
### Creating test file for this demo:
testdata = {
    'Transaction unique identifier' : ['{81B82214-7FBC-4129-9F6B-4956B4A663AD}', '{8046EC72-1466-42D6-A753-4956BF7CD8A2}', 
                                       '{278D581A-5BF3-4FCE-AF62-4956D87691E6}', '{1D861C06-A416-4865-973C-4956DB12CD12}', 
                                       '{DD8645FD-A815-43A6-A7BA-4956E58F1874}'],
    'Price' : [25000, 42500, 45000, 43150, 18899],
    'Date of Transfer' : ['1995-08-18 00:00', '1995-08-09 00:00', '1995-06-30 00:00', '1995-11-24 00:00', '1995-06-23 00:00'],
    'Property Type' : ['T', 'S', 'T', 'T', 'S'],
    'Old/New' : ['N', 'N', 'N', 'N', 'N'],
    'Duration' : ['F', 'F', 'F', 'F', 'F'],
    'Town/City' : ['OLDHAM', 'GRAYS', 'HIGHBRIDGE', 'BEDFORD', 'WAKEFIELD'],
    'District' : ['OLDHAM', 'THURROCK', 'SEDGEMOOR', 'NORTH BEDFORDSHIRE', 'LEEDS'],
    'County' : ['GREATER MANCHESTER', 'THURROCK', 'SOMERSET', 'BEDFORDSHIRE', 'WEST YORKSHIRE'],
    'PPDCategory Type' : ['A', 'A', 'A', 'A', 'A'],
    'Record Status - monthly file only' : ['A', 'A', 'A', 'A', 'A']
}

import pandas as pd
df = pd.DataFrame(testdata, columns=['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type',
                                     'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type',
                                     'Record Status - monthly file only'])
df.to_csv("test_data.csv",index=False)

In [13]:
df

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [14]:
testdata

{'Transaction unique identifier': ['{81B82214-7FBC-4129-9F6B-4956B4A663AD}',
  '{8046EC72-1466-42D6-A753-4956BF7CD8A2}',
  '{278D581A-5BF3-4FCE-AF62-4956D87691E6}',
  '{1D861C06-A416-4865-973C-4956DB12CD12}',
  '{DD8645FD-A815-43A6-A7BA-4956E58F1874}'],
 'Price': [25000, 42500, 45000, 43150, 18899],
 'Date of Transfer': ['1995-08-18 00:00',
  '1995-08-09 00:00',
  '1995-06-30 00:00',
  '1995-11-24 00:00',
  '1995-06-23 00:00'],
 'Property Type': ['T', 'S', 'T', 'T', 'S'],
 'Old/New': ['N', 'N', 'N', 'N', 'N'],
 'Duration': ['F', 'F', 'F', 'F', 'F'],
 'Town/City': ['OLDHAM', 'GRAYS', 'HIGHBRIDGE', 'BEDFORD', 'WAKEFIELD'],
 'District': ['OLDHAM',
  'THURROCK',
  'SEDGEMOOR',
  'NORTH BEDFORDSHIRE',
  'LEEDS'],
 'County': ['GREATER MANCHESTER',
  'THURROCK',
  'SOMERSET',
  'BEDFORDSHIRE',
  'WEST YORKSHIRE'],
 'PPDCategory Type': ['A', 'A', 'A', 'A', 'A'],
 'Record Status - monthly file only': ['A', 'A', 'A', 'A', 'A']}